# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `current_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

**Подключаем библиотеки:**

In [26]:
import numpy as np
import math
import time

**Чтение графа из файла:**

In [27]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

**Ищем вершину-кандидата для следующей итерации локального поиска:**

In [28]:
def find_next(graph, partition_part, first_part, second_part):
    next_vertex = -1
    next_another_vertex = -1 
    now_best = 0
    diff = count_transp(graph, partition_part)
    another_part = graph[0] - partition_part
    for vertex in partition_part - first_part:
        for another_vertex in another_part - second_part:
            fake_edge = False
            if (vertex, another_vertex) in graph[1] or (another_vertex, vertex) in graph[1]:
                fake_edge = True
            now_difference = diff[vertex - 1] + diff[another_vertex - 1] + 2 * fake_edge
            if next_vertex == -1 or now_difference < now_best:
                next_vertex = vertex
                next_another_vertex = another_vertex
                now_best = now_difference
    return next_vertex, next_another_vertex

**Считаем, насколько мы улучшили результат при переносе вершины:**

In [29]:
def count_transp(graph, partition_part):
    differences = np.zeros(len(graph[0]))
    for vertex, another_vertex in graph[1]:
        if (vertex in partition_part) != (another_vertex in
                                         partition_part):
            differences[vertex - 1], differences[another_vertex - 1] = differences[vertex - 1] - 1, differences[another_vertex - 1] - 1
        else:
            differences[vertex - 1], differences[another_vertex - 1] = differences[vertex - 1] + 1, differences[another_vertex - 1] + 1
    return differences

**Ищем сбалансированное разбиение при помощи эвристики Кернигана-Лина:**

In [30]:
def variable_depth_local_search(graph, depths):
    first_part, second_part = set(), set()
    current_point = set(range(1, len(graph[0]) // 2 + 1))
    for depth in range(depths):
        next_vertex, next_another_vertex = find_next(graph, current_point, first_part, second_part)
        current_point.remove(next_vertex)
        current_point.add(next_another_vertex)
        first_part.add(next_vertex)
        second_part.add(next_another_vertex)
    return current_point

**Запускаем тестирование на всех графах:**

In [34]:
def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    depths = [3, 1, 2, 1]
    for filename, depth in zip(filenames, depths):
        instance = read_instance(filename)
        print('Поиск решения для графа {}…\n'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance, depth))
        time_elapsed = time.monotonic()-time_start
        print('Завершено за {:.2} секунд с качеством {}\n\n'.format(time_elapsed, quality))

In [35]:
run_all()

Поиск решения для графа add20.graph…
Завершено за 5.6 секунд с качеством 11112


Поиск решения для графа cti.graph…
Завершено за 1e+02 секунд с качеством 94056


Поиск решения для графа t60k.graph…


KeyboardInterrupt: 

## Выводы
К сожалению подобрать параметры для работы меньше минуты не получилось. Как видно, с такими параметрами качества решений не хуже (иногда лучше) исходных. Как ускорить алгоритм не знаю :(